In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import time
import copy

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0, contrast=0.05, saturation=0.05),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 2)
        self.bnm1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12, 48, 2)
        self.bnm2 = nn.BatchNorm2d(48)
        self.conv3 = nn.Conv2d(48, 192, 3)
        self.bnm3 = nn.BatchNorm2d(192)
        self.conv4 = nn.Conv2d(192, 768, 2)
        self.bnm4 = nn.BatchNorm2d(768)
        self.conv5 = nn.Conv2d(768, 3072, 3)
        self.bnm5 = nn.BatchNorm2d(3072)
        self.fc1 = nn.Linear(4*3072, 480)
        self.fc2 = nn.Linear(480, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.bnm1(x)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.bnm2(x)
        x = F.relu(self.conv3(x))
        x = self.bnm3(x)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.bnm4(x)
        x = F.relu(self.conv5(x))
        x = self.bnm5(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*3072)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = Net().to(device)
model = nn.DataParallel(model)
print(model)

def train(model, device, train_loader, optimizer, epoch,log_interval):
    model.train()
    avg_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # zero the gradient buffers
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step() # Does the update
        avg_loss+=F.nll_loss(output, target, reduction='sum').item()
        
        if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{:5.0f}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
    avg_loss/=len(train_loader.dataset)
    return avg_loss

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print("----------------------------------------------------Best Accuracy was: ", best_acc)
    
            
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss,accuracy

epochs = 60
log_interval = 200
save_model = False
lr=0.03
momentum = 0.5

train_losses = []
test_losses = []
accuracy_list = []
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

best_acc = 0.0
best_model_wts = 0

for epoch in range(1, epochs + 1):
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    if accuracy > best_acc:
        best_acc = accuracy
        best_model_wts = copy.deepcopy(model.state_dict())
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)
    if epoch%9 == 0:
        lr = lr*0.7
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
        torch.save(best_model_wts, str(best_acc)+"CIFAR_cnn.pt")

torch.save(best_model_wts, str(best_acc)+"CIFAR_cnn.pt")

#from google.colab import files
#files.download(str(best_acc)+"CIFAR_cnn.pt")


Files already downloaded and verified
Files already downloaded and verified
cuda:0
DataParallel(
  (module): Net(
    (conv1): Conv2d(3, 12, kernel_size=(2, 2), stride=(1, 1))
    (bnm1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(12, 48, kernel_size=(2, 2), stride=(1, 1))
    (bnm2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(48, 192, kernel_size=(3, 3), stride=(1, 1))
    (bnm3): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(192, 768, kernel_size=(2, 2), stride=(1, 1))
    (bnm4): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv5): Conv2d(768, 3072, kernel_size=(3, 3), stride=(1, 1))
    (bnm5): BatchNorm2d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=12288, out_features=480, bias=True)
    (fc2): Linear(in_features=480, o

KeyboardInterrupt: ignored

In [11]:
for epoch in range(61, 101):
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    if accuracy > best_acc:
        best_acc = accuracy
        best_model_wts = copy.deepcopy(model.state_dict())
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)
    if epoch%3 == 0:
        lr = lr*0.9
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
        torch.save(best_model_wts, str(best_acc)+"CIFAR_cnn.pt")

torch.save(best_model_wts, str(best_acc)+"CIFAR_cnn.pt")

from google.colab import files
files.download(str(best_acc)+"CIFAR_cnn.pt")


Train Epoch: 61 [    0/50000 (0%)]	Loss: 0.108310
Train Epoch: 61 [25600/50000 (51%)]	Loss: 0.028833

Test set: Average loss: 0.4102, Accuracy: 9009/10000 (90.09%)

----------------------------------------------------Best Accuracy was:  90.13
Train Epoch: 62 [    0/50000 (0%)]	Loss: 0.028101
Train Epoch: 62 [25600/50000 (51%)]	Loss: 0.031170

Test set: Average loss: 0.4132, Accuracy: 9011/10000 (90.11%)

----------------------------------------------------Best Accuracy was:  90.13
Train Epoch: 63 [    0/50000 (0%)]	Loss: 0.030517
Train Epoch: 63 [25600/50000 (51%)]	Loss: 0.040700

Test set: Average loss: 0.4191, Accuracy: 8961/10000 (89.61%)

----------------------------------------------------Best Accuracy was:  90.13
Train Epoch: 64 [    0/50000 (0%)]	Loss: 0.081080
Train Epoch: 64 [25600/50000 (51%)]	Loss: 0.042507

Test set: Average loss: 0.4163, Accuracy: 8997/10000 (89.97%)

----------------------------------------------------Best Accuracy was:  90.13
Train Epoch: 65 [    0/50000

KeyboardInterrupt: ignored

In [12]:
for epoch in range(101, 140):
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    if accuracy > best_acc:
        best_acc = accuracy
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, str(best_acc)+"CIFAR_cnn.pt")

    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)
    lr = lr*0.9
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

torch.save(best_model_wts, str(best_acc)+"CIFAR_cnn.pt")


Train Epoch: 101 [    0/50000 (0%)]	Loss: 0.028466
Train Epoch: 101 [25600/50000 (51%)]	Loss: 0.014414

Test set: Average loss: 0.4223, Accuracy: 9030/10000 (90.30%)

----------------------------------------------------Best Accuracy was:  90.38
Train Epoch: 102 [    0/50000 (0%)]	Loss: 0.057371
Train Epoch: 102 [25600/50000 (51%)]	Loss: 0.014700

Test set: Average loss: 0.4249, Accuracy: 9037/10000 (90.37%)

----------------------------------------------------Best Accuracy was:  90.38
Train Epoch: 103 [    0/50000 (0%)]	Loss: 0.022903
Train Epoch: 103 [25600/50000 (51%)]	Loss: 0.038862

Test set: Average loss: 0.4261, Accuracy: 9034/10000 (90.34%)

----------------------------------------------------Best Accuracy was:  90.38
Train Epoch: 104 [    0/50000 (0%)]	Loss: 0.036450
Train Epoch: 104 [25600/50000 (51%)]	Loss: 0.044750

Test set: Average loss: 0.4248, Accuracy: 9041/10000 (90.41%)

----------------------------------------------------Best Accuracy was:  90.38
Train Epoch: 105 [  

In [23]:
"""----------------CALCULATE ACCURACY, PRECISION, RECALL, CONFUSION MATRIX----------------------"""

model.load_state_dict(torch.load("90.5CIFAR_cnn.pt"))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
y_true = []
y_pred = []
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(1000):
            label = labels[i]
            y_true.append(classes[labels[i]])
            y_pred.append(classes[predicted[i]])
            class_correct[label] += c[i].item()
            class_total[label] += 1


import sklearn.metrics as metr
mtrx = metr.confusion_matrix(y_true, y_pred, labels=classes)
print("Confusion matrix: \n", mtrx)

precisions = [0,0,0,0,0,0,0,0,0,0]
recalls = [0,0,0,0,0,0,0,0,0,0]
f1_scores = [0,0,0,0,0,0,0,0,0,0]
for i in range(10):
    suma_row=0
    suma_column=0
    for j in range(10):
        suma_row += mtrx[i][j]
        suma_column += mtrx[j][i]
    precisions[i] = mtrx[i][i]/(suma_row)*100
    recalls[i] = mtrx[i][i]/(suma_column)*100
    f1_scores[i] = 2 * (precisions[i] * recalls[i])/(precisions[i] + recalls[i])
    print('Accuracy of %5s : %2d %%  Precision: %4s %%  Recall: %4d %%  F1-score: %4s %%'% (classes[i],
                                                                                            100 * class_correct[i] / class_total[i],
                                                                                            precisions[i],
                                                                                            recalls[i],
                                                                                            f1_scores[i]))


Confusion matrix: 
 [[913   7  17  14   3   7   1   5  18  15]
 [  6 936   1   3   0   1   0   3   4  46]
 [ 24   1 831  37  39  21  27  10   2   8]
 [ 14   2  25 746  32 107  27  20   7  20]
 [ 12   0  36  23 851  16  23  32   1   6]
 [  7   2  11  88  25 830   7  17   2  11]
 [  5   0  22  29  13   9 913   2   1   6]
 [  8   1   9  21  21  33   2 894   1  10]
 [ 43  16   9   3   3   1   3   1 904  17]
 [  8  28   1   4   1   1   1   2  12 942]]
Accuracy of plane : 91 %  Precision: 91.3 %  Recall:   87 %  F1-score: 89.50980392156863 %
Accuracy of   car : 93 %  Precision: 93.60000000000001 %  Recall:   94 %  F1-score: 93.92875062719519 %
Accuracy of  bird : 83 %  Precision: 83.1 %  Recall:   86 %  F1-score: 84.70948012232417 %
Accuracy of   cat : 74 %  Precision: 74.6 %  Recall:   77 %  F1-score: 75.81300813008131 %
Accuracy of  deer : 85 %  Precision: 85.1 %  Recall:   86 %  F1-score: 85.61368209255532 %
Accuracy of   dog : 83 %  Precision: 83.0 %  Recall:   80 %  F1-score: 81.9348469